Вы успешно создали полноценную торговую стратегию на основе модели машинного обучения и обеспечили фиксацию и сравнение метрик полученных моделей.

Вы решаете построить более сложные модели машинного обучения и хотите использовать нейросетевые модели, в том числе для обработки временных рядов и глубокие нейронные сети. Здесь вам могут помочь методы рекуррентных сетей и современные трансформерные архитектуры.

Поговорив с коллегами, вы понимаете, что самостоятельно построить и обучить действительно сложные архитектуры будет сложно и решаете воспользоваться предобученными свободно распространяемыми моделями.

На основании вышесказанного вам необходимо построить несколько моделей на основе нейронных сетей, позволяющих прогнозировать оптимальное торговое действие.

На основе представленной информации, вам предлагается:

Создать модель (торговую стратегию) на основе нейронных сетей для прогнозирования оптимального торгового действия. Можно использовать, как самостоятельно обученные архитектуры, так и использовать предобученные сети или фреймворки.
Провести тестирование разработанной стратегии на валидационном датасете.
Зафиксировать метрики модели для дальнейшего сравнения экспериментов.
Сформировать дашборд, показывающий эффективность различных торговых
стратегий.

## Setup and Imports
Install required packages and import dependencies

In [ ]:
%pip install backtesting nbformat matplotlib numpy pandas scikit-learn torch tqdm --quiet

In [ ]:
import os
import traceback
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from assets.DataProvider import DataProvider
from assets.FeaturesGenerator import FeaturesGenerator
from assets.hw5.HybridStrategySMA_ML import HybridStrategySMA_ML
from assets.hw5.Models import StockCNN, StockCNN_LSTM, StockGRU, StockLSTM
from assets.hw5.SMABaselineGenerator import SMABaselineGenerator
from backtesting import Backtest, Strategy

## Helper Functions
Define utility functions for sequence creation, model training, and testing

In [ ]:
def create_sequences(data_values, target_values, window_size):
    """Create sequences for time series data"""
    X, y = [], []
    for i in range(len(data_values) - window_size):
        X.append(data_values[i:(i + window_size)])
        y.append(target_values[i + window_size])
    return np.array(X), np.array(y)

In [ ]:
def train_model(title, model, dataloader, num_epochs=10):
    """Train a PyTorch model"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    model.eval()
    return model

In [ ]:
def test_model(title, model, dataloader):
    """Test a trained model and calculate accuracy"""
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.numpy())
            all_targets.extend(targets.numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"{title} Accuracy: {accuracy:.4f}")
    return accuracy
    # Plot actual vs predicted labels
    plt.figure(figsize=(12, 6))
    plt.plot(all_targets[-30:], label='Actual', color='blue', linestyle='--', alpha=0.7)
    plt.plot(all_predictions[-30:], label='Predicted', color='red', alpha=0.7)
    plt.title(title)
    plt.xlabel("Time Step")
    plt.ylabel("Signal (0=Sell, 1=Hold, 2=Buy)")
    plt.legend()
    plt.show()
    
    return accuracy

## Main Trading Strategy Implementation

In [ ]:
ticker = 'BTC/USDT'
timeframe = '1h'

# Initial parameters
strategy_best_params = {}
threshold = 0.2

In [ ]:
# 1. Load the data
data_provider = DataProvider(tickers=[ticker], skip_dashboard=True)
if ticker not in data_provider.data:
    if not data_provider.data_load():
        data_provider.data_request()
        data_provider.data_save()
    data_provider.clean_data()
data = data_provider.data[ticker]

In [ ]:
# 2. Generate features
features_generator = FeaturesGenerator()
data_initial_ohlc = data.copy()
feature_matrix, feature_names = features_generator.prepare_features(data_initial_ohlc)
num_features = feature_matrix.shape[1]
print(f"Number of features generated: {num_features} ({len(feature_names)} names)")

In [ ]:
# 3. Generate target using SMA strategy
strategy_hw2 = SMABaselineGenerator(ticker, timeframe, data_provider)
strategy_best_results, strategy_best_params, strategy_trades = strategy_hw2.evaluate_strategies()

In [ ]:
# 4. Prepare data for ML
data_extended = feature_matrix.copy()
if strategy_trades is None or strategy_trades.empty:
    print("WARN: No trades from SMA strategy. PnL and ReturnPct will be zero for y_base generation.")
    data_extended['PnL'] = 0.0
    data_extended['ReturnPct'] = 0.0
else:
    strategy_trades = strategy_trades.set_index('EntryTime')
    data_extended = data_extended.merge(strategy_trades[['PnL', 'ReturnPct']], 
                                        left_index=True, right_index=True, how='left')
    data_extended[['PnL', 'ReturnPct']] = data_extended[['PnL', 'ReturnPct']].fillna(value=0)

trades_return_pct = data_extended['ReturnPct'].values
y_base = [1 if x > threshold else 0 for x in trades_return_pct]

In [ ]:
# 5. Create sequences for time series models
window_size = 30
X_np, y_np = create_sequences(feature_matrix.values, np.array(y_base), window_size)
print(f"Sequence shapes - X: {X_np.shape}, y: {y_np.shape}")

In [ ]:
# 6. Prepare PyTorch datasets
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.long)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# 7. Initialize and train models
models = {
    'CNN': {'model': StockCNN(input_channels=num_features, window_size=window_size)},
    'LSTM': {'model': StockLSTM(input_size=num_features)},
    'GRU': {'model': StockGRU(input_size=num_features)},
    'CNN_LSTM': {'model': StockCNN_LSTM(input_channels=num_features)}
}

# Train and test each model
for model_key, model_dict in models.items():
    print(f"\nTraining {model_key}...")
    models[model_key]['model_trained'] = train_model(model_key, model_dict['model'], dataloader)
    models[model_key]['accuracy'] = test_model(model_key, model_dict['model_trained'], dataloader)

In [ ]:
# 8. Backtest hybrid strategies
all_backtest_results = {}
for model_key, model_dict in models.items():
    print(f"\n\n\nPreparing backtest for model: {model_key}...")
    print(f"Running backtest for {model_key} using SMA logic (model not integrated yet).")
    bt = Backtest(data.copy(),  # Pass data.copy()
                    HybridStrategySMA_ML, 
                    cash=10000000, 
                    commission=.002)
    try:
        # Pass sma_short and sma_long to bt.run()
        stats = bt.run(
            sma_short=strategy_best_params['sma_short'],
            sma_long=strategy_best_params['sma_long'],
            model=models[model_key]['model_trained']
        )
        print(f"--- Backtest Results for {model_key} driven strategy ---")
        print(stats)
        all_backtest_results[model_key] = {'stats': stats}
        plot_path = "backtest_plots"
        os.makedirs(plot_path, exist_ok=True)
        plot_filename = os.path.join(plot_path, f"backtest_plot_{model_key}.html")
        try:
            bt.plot(filename=plot_filename, open_browser=False)
            all_backtest_results[model_key]['plot_file'] = plot_filename
            print(f"Backtest plot for {model_key} saved to {plot_filename}")
        except Exception as e:
            print(f"ERROR: Could not generate plot for {model_key}: {e}")
            all_backtest_results[model_key]['plot_file'] = None
            
    except Exception as e:
        print(f"ERROR during backtest for {model_key}: {e}")
        print("This might be due to HW2Strategy_SMA not being fully adapted for model-based trading,")
        print("or issues with data slicing/feature access within the strategy's next() method.")
        traceback.print_exc() # Print detailed traceback

In [ ]:
# Print summary
print("\n\n--- Summary of All Backtest Results ---")

print("\n\n--- Original strategy stats ---")
print(strategy_best_results)

for model_key, strategy_results in all_backtest_results.items():
    print(f"\n--- Results for {model_key} ---")
    print(strategy_results['stats'])
    if 'plot_file' in strategy_results and strategy_results['plot_file']:
        print(f"Plot saved to: {strategy_results['plot_file']}")
    else:
        print("Plot generation failed or was not attempted.")

In [ ]:
def plot_strategy_comparisons(strategy_best_results, all_backtest_results):
    # Extract key metrics for comparison
    metrics = {
        'Return [%]': [],
        'Win Rate [%]': [],
        'Sharpe Ratio': [],
        'Max Drawdown [%]': []
    }
    strategy_names = ['SMA_Base'] + list(all_backtest_results.keys())
    
    # Add baseline SMA results
    metrics['Return [%]'].append(strategy_best_results['Return [%]'])
    metrics['Win Rate [%]'].append(strategy_best_results['Win Rate [%]'])
    metrics['Sharpe Ratio'].append(strategy_best_results['Sharpe Ratio'])
    metrics['Max Drawdown [%]'].append(strategy_best_results['Max. Drawdown [%]'])
    
    # Add hybrid strategy results
    for model_key in all_backtest_results.keys():
        stats = all_backtest_results[model_key]['stats']
        metrics['Return [%]'].append(stats['Return [%]'])
        metrics['Win Rate [%]'].append(stats['Win Rate [%]'])
        metrics['Sharpe Ratio'].append(stats['Sharpe Ratio'])
        metrics['Max Drawdown [%]'].append(stats['Max. Drawdown [%]'])
    
    # Create subplots for each metric
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Strategy Performance Comparison', fontsize=16)
    
    for (metric, values), ax in zip(metrics.items(), axes.flat):
        colors = ['blue'] + ['green' if v > values[0] else 'red' for v in values[1:]]
        if metric in ['Max Drawdown [%]']:  # Inverse colors for metrics where lower is better
            colors = ['blue'] + ['green' if v < values[0] else 'red' for v in values[1:]]
            
        bars = ax.bar(strategy_names, values, color=colors)
        ax.set_title(metric)
        ax.set_xticklabels(strategy_names, rotation=45)
        
        # Add value labels on top of each bar
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Create a summary table
    summary_df = pd.DataFrame(metrics, index=strategy_names)
    display(summary_df)

# Plot the comparisons
plot_strategy_comparisons(strategy_best_results, all_backtest_results)